In [ ]:
import pandas as pd
import numpy as np
import torch
import io
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import time
from sklearn.metrics import f1_score


In [ ]:
%ls
prefix = '/data1/tjss/'
# prefix = '/content/drive/My Drive/Summer research /'
doc_level_df = pd.read_csv(prefix+'data/msgs_tec.csv')
word_level_df = pd.read_csv(prefix+'data/NRC_ht_emotion_sentiment.csv')

=0.0.7            Miniconda3-latest-Linux-x86_64.sh          test/
examples.desktop  nltk_data/                                 tut3-model.pt
jupyter           NRC_Emotion_Classification_DeepShap.ipynb
miniconda3/       PyTorch_Classification__New.ipynb


/home/tjss/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#EDA
print(doc_level_df.shape)
print(doc_level_df.emotion.unique())
doc_level_df = doc_level_df.dropna()
print(doc_level_df.shape)
doc_level_df['message'] = doc_level_df['message'].str.lower() 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
doc_level_df['emotion_code'] = lb_make.fit_transform(doc_level_df['emotion'])
print(doc_level_df.iloc[:10,:])
new_doc_level_df = doc_level_df[['message', 'emotion_code']]
print(new_doc_level_df.iloc[:10,:])
new_doc_level_df.head(10)

new_doc_level_df.loc[(new_doc_level_df.emotion_code!=3),'emotion_code']=0
new_doc_level_df.loc[(new_doc_level_df.emotion_code==3),'emotion_code']=1

new_doc_level_train, new_doc_level_test = train_test_split(new_doc_level_df, test_size=0.2, random_state=1)
new_doc_level_train, new_doc_level_val = train_test_split(new_doc_level_train, test_size=0.25, random_state=1)

(21051, 3)
['surprise' 'sadness' 'joy' 'disgust' 'fear' 'anger' nan]
(21049, 3)
           message_id                                            message  \
0  145353048817012736  thinks that @melbahughes had a great 50th birt...   
1  144279638024257536  como una expresiã³n tan simple, una sola oraci...   
2  140499585285111809  the moment when you get another follower and y...   
3  145207578270507009  be the greatest dancer of your life! practice ...   
4  139502146390470656  eww.. my moms starting to make her annual rum ...   
5  146042696899887106  if ur heart hurts all the time for tht person ...   
6  145492569609084928  i feel awful, and it's way too freaking early....   
7  145903955229151232  so chuffed for safc fans! bet me dar comes in ...   
8  142717613234069504  making art and viewing art are different at th...   
9  144183822873927680  soooo dooowwwn!! move on, get some sleep... me...   

    emotion  emotion_code  
0  surprise             5  
1   sadness             4  

/home/tjss/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [ ]:
print(new_doc_level_train.emotion_code.value_counts())
print(new_doc_level_val.emotion_code.value_counts())
print(new_doc_level_test.emotion_code.value_counts())

0    7706
1    4923
Name: emotion_code, dtype: int64
0    2570
1    1640
Name: emotion_code, dtype: int64
0    2534
1    1676
Name: emotion_code, dtype: int64


In [ ]:
print(new_doc_level_train.shape)
print(new_doc_level_val.shape)
print(new_doc_level_test.shape)

(12629, 2)
(4210, 2)
(4210, 2)


In [ ]:
# 0-> not Joy
# 1-> Joy


# 0 Anger
# 1 Disgust
# 2 Fear
# 3 Joy
# 4 Sadness
# 5 Surprise

In [ ]:
new_doc_level_train.to_csv(prefix+"data/new_doc_level_train.csv")
new_doc_level_val.to_csv(prefix+"data/new_doc_level_val.csv")
new_doc_level_test.to_csv(prefix+"data/new_doc_level_test.csv")

In [ ]:
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data


In [ ]:
embeddings = load_vectors(prefix+"data/crawl-300d-2M.vec")

In [ ]:
print(type(embeddings))
print(len(embeddings))
for word in embeddings:
  embeddings[word] = [float(i) for i in embeddings[word]]
embeddings_keylist = list(embeddings)
embeddings_vectorlist = list(embeddings.values())
word_to_index={}
index_to_word={}
for i in range(len(embeddings_keylist)):
  word_to_index[embeddings_keylist[i]]=i
  index_to_word[i]=embeddings_keylist[i]

<class 'dict'>
1999995


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# from spacy.lang.en import English
# nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
# tokenizer = nlp.Defaults.create_tokenizer(nlp)



[nltk_data] Downloading package punkt to /home/tjss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# # Dummy dataloader
# test_df = pd.read_csv(prefix+"data/new_doc_level_val.csv")
# print(test_df.shape)
# test_df = test_df.iloc[:,1:]
# curr_tex = test_df.iloc[6,0]
# curr_emot = test_df.iloc[6,1]
# curr_embed = np.zeros((300))
# print("curr text", curr_tex)
# print(curr_embed)
# tot_cnt=0
# ign_cnt=0
# for word in word_tokenize(curr_tex):
#   print(word)
#   tot_cnt+=1
#   if word in embeddings.keys():
#     curr_embed = curr_embed+embeddings[word]
#   else:
#     ign_cnt+=1
# print("percentage lost", ign_cnt/tot_cnt*100, "%")
# print(curr_embed.shape)
# curr_embed = curr_embed
# embed_t = torch.from_numpy(curr_embed)
# print(embed_t)

In [ ]:
# device = "cuda" if torch.cuda.is_available() else "cpu"

# kwargs = {'num_workers': 1, 'pin_memory': True} if device=='cuda' else {}

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
class FastTextVectorLoader(Dataset):
  def __init__(self, csv_file, embeddings):
    self.frame = pd.read_csv(csv_file)
    self.embeddings = embeddings
  def __len__(self):
    return len(self.frame)
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()
    curr_tex = self.frame.iloc[idx,1]
    curr_emot = self.frame.iloc[idx,2]
    curr_embed = np.zeros((300))
    tot_cnt=0
    ign_cnt=0
    
    for word in word_tokenize(curr_tex):
      tot_cnt+=1
      if word in embeddings.keys():
        # print(word)
        curr_embed = curr_embed+embeddings[word]
      else:
        ign_cnt+=1
    # print("percentage lost", ign_cnt/tot_cnt*100, "%")
    curr_embed = curr_embed
    embed_t = torch.from_numpy(curr_embed)
    # embed_t = embed_t.to(torch.device("cuda:0"))
    # curr_emot = curr_emot.cuda()
    data = {'vector':embed_t, 'label':curr_emot}
    return data

In [ ]:
train_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_train.csv", embeddings)
val_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_val.csv", embeddings)
test_dataset = FastTextVectorLoader(prefix+"data/new_doc_level_test.csv", embeddings)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=64,shuffle=True, num_workers=1)
val_dataloader = DataLoader(val_dataset, batch_size=64,shuffle=True, num_workers=1)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=1)


In [ ]:
# testing dataloader
count = 0
for i,batch_data in enumerate(train_dataloader):
  count += 1
  print(batch_data['vector'])
  print(batch_data['vector'].size())
  print(batch_data['label'])
  # batch_data['vector'].cuda()
  # batch_data['vector'].to(device)
  print(batch_data['vector'].is_cuda)
  print(batch_data['label'].is_cuda)
  # print(text)
  # print(text_length)
  if count == 1:
    break


tensor([[-0.9043, -0.7394,  0.6312,  ..., -0.7097, -0.8393,  0.1911],
        [-0.6624, -1.0909, -0.6750,  ..., -0.2373, -1.3354, -0.8354],
        [-1.4365,  0.1660, -0.9491,  ..., -0.6556, -1.0353,  0.2139],
        ...,
        [ 0.6464, -1.4903, -0.1685,  ..., -0.4613, -1.4939,  0.3455],
        [-0.9206, -0.2830,  0.0548,  ..., -0.4794, -0.0606, -0.9066],
        [-0.7463, -1.1811, -0.1984,  ..., -0.5583, -0.5830,  0.0896]],
       dtype=torch.float64)
torch.Size([64, 300])
tensor([0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0])
False
False


In [ ]:
 def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        text_vector = batch['vector']
        # text_vector = batch['vector'].to(device)
        predictions = model(text_vector)
        
        y_pred_tag = torch.round(torch.sigmoid(predictions))
        
        # labels = batch['label'].to(device)
        labels = batch['label']
        labels = labels.float()
        labels = labels.reshape([ len(labels),1])
        loss = criterion(predictions, labels)
        
        f1 = f1_score(labels.detach().numpy(), y_pred_tag.detach().numpy())
        acc = calc_accuracy(y_pred_tag, labels)


        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text_vector = batch['vector']
            # text_vector = batch['vector'].to(device)
            predictions = model(text_vector)
            y_pred_tag = torch.round(torch.sigmoid(predictions))
            labels = batch['label']
            # labels = batch['label'].to(device)
            labels = labels.float()
            labels = labels.reshape([len(labels),1])
            loss = criterion(predictions, labels)
            f1 = f1_score(labels, y_pred_tag)
            acc = calc_accuracy(y_pred_tag, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def calc_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    # rounded_preds = torch.round(torch.sigmoid(preds))
    
    correct = (preds == y).float() #convert into float for division 
    # print(correct)
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, embedding_length, output_dim):        
        super().__init__()
        
        self.embedding_length = embedding_length
        
        self.output_dim = output_dim

        # self.embedding = nn.Embedding(self.vocab_size, self.embedding_length, padding_idx=pad_idx)
        # self.embedding.weight = nn.Parameter(word_embeddings, requires_grad = False)
        self.fc1 = nn.Linear(self.embedding_length, 512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.4)
        
    def forward(self, text):        
        pooled_drop_op = self.dropout1(text)
        fc1_op = self.fc1(pooled_drop_op)
        fc1_drop_op = self.dropout2(fc1_op)
        fc2_op = self.fc2(fc1_drop_op)
        fc2_drop_op = self.dropout2(fc2_op) 
        fc3_op = self.fc3(fc2_drop_op)       
        return self.fc4(fc3_op)
        # text = self.relu(self.fc1(text))
        # text = self.relu(self.fc2(text))
        # text = self.relu(self.fc3(text))
        # text = self.dropout1(text)
        # return self.fc4(text)
        

In [ ]:
EMBEDDING_DIM = 300
OUTPUT_DIM = 1
model = FastText( EMBEDDING_DIM,OUTPUT_DIM)
model = model.double()

In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.BCEWithLogitsLoss()
# criterion = criterion.to(device)

In [ ]:
N_EPOCHS =11
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc, train_f1 = train(model, train_dataloader, optimizer, criterion)
    valid_loss, valid_acc, val_f1 = evaluate(model, val_dataloader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Train F1:{train_f1}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val F1:{val_f1}')

Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.662 | Train Acc: 60.90% | Train F1:0.0
	 Val. Loss: 0.633 |  Val. Acc: 62.30% | Val F1:0.0
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.624 | Train Acc: 65.72% | Train F1:0.4
	 Val. Loss: 0.608 |  Val. Acc: 68.23% | Val F1:0.30769230769230765
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.602 | Train Acc: 69.50% | Train F1:0.28571428571428575
	 Val. Loss: 0.588 |  Val. Acc: 70.87% | Val F1:0.33333333333333337
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.582 | Train Acc: 71.37% | Train F1:0.6153846153846153
	 Val. Loss: 0.573 |  Val. Acc: 71.80% | Val F1:0.5263157894736842
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.566 | Train Acc: 72.38% | Train F1:0.7058823529411764
	 Val. Loss: 0.561 |  Val. Acc: 72.92% | Val F1:0.7058823529411764
Epoch: 06 | Epoch Time: 0m 13s
	Train Loss: 0.558 | Train Acc: 73.21% | Train F1:0.7999999999999999
	 Val. Loss: 0.552 |  Val. Acc: 73.29% | Val F1:0.6842105263157896
Epoch: 07 | Epoch Time: 0m 13s
	Train Lo

In [ ]:

test_loss, test_acc, test_f1 = evaluate(model, test_dataloader, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1}')

Test Loss: 0.521 | Test Acc: 76.06% | Test F1: 0.5161290322580646


In [ ]:
torch.save(model.state_dict(),prefix+'data/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy Reviews)Shap.pt' )

In [ ]:
model1 = FastText(EMBEDDING_DIM,OUTPUT_DIM)
model1.load_state_dict(torch.load(prefix+'data/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy Reviews)Shap.pt'))
model1.eval()

FastText(
  (fc1): Linear(in_features=300, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.4, inplace=False)
)

In [ ]:
%pip install shap
import shap

Note: you may need to restart the kernel to use updated packages.


In [ ]:
test_df = pd.read_csv(prefix+"data/new_doc_level_train.csv")
test_df.head(5)

,Unnamed: 0,message,emotion_code
0,11390,"io domani volevo vestirmi tutta figa, ma il mi...",0
1,1136,"churchhhh! then exam stuff, and making 30 dang...",1
2,9805,empty fuel tank & a $ load of traffic,1
3,16383,he who cannot forgive others destroys the brid...,1
4,12353,hoje meu dia vai ser study study and study. te...,0


In [ ]:
def get_word_embeddings(dataframe_csv, idx_limit):
  vocab={}
  bg = torch.empty((0,300))
  df = pd.read_csv(dataframe_csv)
  for idx, row in df.iterrows():
    curr_text = df.iloc[idx,1]
    curr_label = df.iloc[idx,2]
    if(curr_label==1):
      for word in word_tokenize(curr_text):
        if(word in embeddings.keys() and word not in vocab.keys()):
          vocab[word]=0
          curr_embed = torch.from_numpy(np.array(embeddings[word]))
          curr_embed = curr_embed.reshape((1, len(curr_embed)))
          bg = torch.cat((bg, curr_embed))
    if(idx==idx_limit):
      break

  return bg, vocab


In [ ]:
new_bg,_ = get_word_embeddings(prefix+"data/new_doc_level_train.csv", 1500)
print(new_bg.size())

torch.Size([2312, 300])


In [ ]:
e = shap.DeepExplainer(model, new_bg)

In [ ]:
explain_examples, vocab1  = get_word_embeddings(prefix+"data/new_doc_level_test.csv", 1200)
print(explain_examples.size())

torch.Size([2212, 300])


In [ ]:
st = time.time()
shap_vals = e.shap_values(explain_examples.float())
print(time.time()-st)

944.3869395256042


In [ ]:
print(len(shap_vals))

2212


In [ ]:
print(len(vocab1))

2212


In [ ]:

print((shap_vals).shape)

(2212, 300)


In [ ]:
shap_means_joy = shap_vals.mean(axis=1)

In [ ]:
idx=0
for word in vocab1:
  vocab1[word]=shap_means_joy[idx]
  idx+=1

In [ ]:
for word in vocab1:
  print(word, " ", vocab1[word])

ed   -0.00029132182004833245
balls   -0.0007802414226404038
says   -0.0011258309401901807
antiques   0.00047031097921984696
roadshow   -4.787479596554748e-06
makes   -0.0002242164958754426
him   -0.00046831257148584066
cry   1.3534775597711772e-05
.   0.00010554980385674715
a   2.4812586155904768e-05
victory   2.2415618555151572e-05
at   0.00020109264370575
old   -0.00029751064044442125
trafford   -0.0002621597453769776
is   -0.00039486199244308386
my   0.00034923679057127945
tipping   -0.000622902400259597
point   -0.00043334968955280567
i   -0.0005596285018589198
officially   -0.000284716933790009
got   -0.000638128526155974
glasses   1.474868334661141e-05
,   -7.242196518999457e-05
again   0.00014919096953821108
time   0.0005356842243185888
for   0.00031150303167828286
the   7.561363041091199e-05
you   -0.0003037768718781081
look   -0.00040108858278939093
like   -0.0006354138241062046
librarian   -0.0003077754834086221
!   0.0001482541615364738
comments   -0.0010113105469451375
-.- 

In [ ]:
df = pd.DataFrame(list(vocab1.items()),columns = ['column1','column2']) 

In [ ]:
lexicon_df_joy = df.sort_values(by='column2')

In [ ]:
lexicon_df_joy.head(50)

,column1,column2
1743,blindsided,-0.002715
1067,scared,-0.002582
1189,eww,-0.002513
585,broek,-0.002501
590,unrespectful,-0.002397
405,hater,-0.002341
742,cougar,-0.002239
1536,ugly,-0.002129
1065,react,-0.002127
620,shocking,-0.002049


In [ ]:
lexicon_df_joy.tail(50)

,column1,column2
771,laundry,0.001672
541,xmas,0.001674
462,peace,0.001678
2175,knackered,0.001707
129,thankful,0.001732
218,hour,0.001734
1617,afternoon,0.001737
1703,singing,0.001743
134,happiness,0.001781
811,holiday,0.001795


In [ ]:
lexicon_df_joy.to_csv(prefix+"results/SummedEmbeddingsFasttext_NRCJoyVsNotJoy_wordLevelBackground(Joy Reviews)Shap.csv")
